In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
import torch.hub
from torchvision.models import efficientnet_b3,densenet161, EfficientNet_B3_Weights, DenseNet161_Weights, resnet152, ResNet152_Weights, inception_v3, vgg16, VGG16_Weights, vgg19, VGG19_Weights,Inception_V3_Weights
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from PIL import Image
import os
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau

# Dataset definition

In [2]:
class ImageAuthenticityDataset(Dataset):
    """Dataset for image quality assessment."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with annotations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.dir_path = os.path.dirname(csv_file)  # Directory of the CSV file

    def __len__(self):
        """Returns the number of samples in the dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieves an image and its labels by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple (image, labels) where:
                image (PIL.Image): The image.
                labels (torch.Tensor): Tensor containing quality and authenticity scores.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # TODO: to be fixed, right now is folder dependent
        # Assuming your CSV has the relative path in the 4th column (index 3)
        # And you need to adjust the path to be relative to where you run the script
        img_name = self.data.iloc[idx, 3].replace("./", "../")
        image = Image.open(img_name).convert('RGB')
        
        # Assuming authenticity is in the 2nd column (index 1) and is a float score
        authenticity = self.data.iloc[idx, 1]  
        # Note: Your dataset returns a float tensor of shape [1]. 
        # This is suitable for regression tasks or specific binary setups.
        # For standard binary classification (e.g., with CrossEntropyLoss), 
        # you might need an integer tensor (0 or 1). Adjust if needed.
        labels = torch.tensor([authenticity], dtype=torch.float) 


        if self.transform:
            image = self.transform(image)

        return image, labels

# Models definitions

In [3]:
class BarlowTwinsAuthenticityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained BarlowTwins ResNet50 instead of ResNet-152
        barlow_twins_resnet = torch.hub.load('facebookresearch/barlowtwins:main', 'resnet50')
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in barlow_twins_resnet.parameters():
                param.requires_grad = False
                
        self.features = nn.Sequential(*list(barlow_twins_resnet.children())[:-2])
        self.avgpool = barlow_twins_resnet.avgpool
        
        
        self.regression_head = nn.Sequential(
                nn.Linear(2048, 512),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(128, 1)
            )    
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        predictions = self.regression_head(x)
        return predictions, x 
    
class EfficientNetB3AuthenticityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained VGG16
        efficent_net = efficientnet_b3(weights=EfficientNet_B3_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in efficent_net.features.parameters():
                param.requires_grad = False
                
        # Extract features up to fc2
        self.features = efficent_net.features
        self.avgpool = efficent_net.avgpool
        
        
        # New regression head for EfficientNet
        self.regression_head = nn.Sequential(
            nn.Linear(1536, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1)  # Predict authenticity
        )
        
    def forward(self, x):
        # Pass through the backbone features
        x = self.features(x)
        # Apply pooling
        x = self.avgpool(x)
        # Flatten the features
        features = torch.flatten(x, 1)
        # Pass through regression head
        predictions = self.regression_head(features)
        
        return predictions, features
    
class DenseNet161AuthenticityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained DenseNet-161
        densenet = densenet161(weights=DenseNet161_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in densenet.parameters():
                param.requires_grad = False
                
        # Store the features
        self.features = densenet.features
        
        # DenseNet already includes a ReLU and pooling after features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # DenseNet-161's output feature dimension is 2208 instead of 2048
        self.regression_head = nn.Sequential(
                nn.Linear(2208, 512),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(128, 1)
            )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        predictions = self.regression_head(x)
        return predictions, x  # Return predictions and features
    
class ResNet152AuthenticityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained ResNet-152 instead of VGG16
        resnet = resnet152(weights=ResNet152_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in resnet.parameters():
                param.requires_grad = False
                
        # Store the backbone (excluding the final fc layer)
        self.features = nn.Sequential(*list(resnet.children())[:-2])
        self.avgpool = resnet.avgpool
        
        self.regression_head = nn.Sequential(
                nn.Linear(2048, 512),
                nn.ReLU(),
                nn.Dropout(0.5),  # Reduced dropout ratio
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(128, 1)
            )    
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        predictions = self.regression_head(x)
        return predictions, x 
        
class VGG16AuthenticityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained VGG16
        vgg = vgg16(weights=VGG16_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in vgg.features.parameters():
                param.requires_grad = False
                
        # Extract features up to fc2
        self.features = vgg.features
        self.avgpool = vgg.avgpool
        self.fc1 = vgg.classifier[:-1]  # Up to fc2 (4096 -> 128)
        
        # New regression head
        self.regression_head = nn.Sequential(
            nn.Linear(4096, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),  
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        features = self.fc1(x)
        predictions = self.regression_head(features)
        return predictions, features
    
class VGG19AuthenticityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained VGG16
        vgg = vgg19(weights=VGG19_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in vgg.features.parameters():
                param.requires_grad = False
                
        # Extract features up to fc2
        self.features = vgg.features
        self.avgpool = vgg.avgpool
        self.fc1 = vgg.classifier[:-1]  # Up to fc2 (4096 -> 128)
        
        # New regression head
        self.regression_head = nn.Sequential(
            nn.Linear(4096, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),  
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        features = self.fc1(x)
        predictions = self.regression_head(features)
        return predictions, features
    
class InceptionV3AuthenticityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained InceptionV3 instead of ResNet152
        inception = inception_v3(weights=Inception_V3_Weights.DEFAULT)
        
        # Disable auxiliary outputs for inference
        inception.aux_logits = False
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in inception.parameters():
                param.requires_grad = False
        
        # Store the backbone (excluding the final fc layer)
        # InceptionV3 structure is different from ResNet, so we need to adapt
        self.features = nn.Sequential(
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2),
            inception.Conv2d_3b_1x1,
            inception.Conv2d_4a_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2),
            inception.Mixed_5b,
            inception.Mixed_5c,
            inception.Mixed_5d,
            inception.Mixed_6a,
            inception.Mixed_6b,
            inception.Mixed_6c,
            inception.Mixed_6d,
            inception.Mixed_6e,
            inception.Mixed_7a,
            inception.Mixed_7b,
            inception.Mixed_7c
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.regression_head = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.5),  
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1)
        )    
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        predictions = self.regression_head(x)
        return predictions, x  # Return predictions and features

# Test function

In [ ]:
def test_model(model, dataloader, device='cpu'):
    """
    Tests the model on the test dataset.

    Args:
        model (nn.Module): The trained model.
        dataloader (DataLoader): The test data loader.
        criterion (nn.Module): The loss function.
        device (str): Device to use for testing ('cuda' or 'cpu'). Defaults to 'cuda'.

    Returns:
        dict: A dictionary containing average loss, PLCC, and SRCC metrics.
    """
    model.eval()  # Set the model to evaluation mode
    model.to(device)
    running_loss = 0.0
    all_predictions = []
    all_labels = []
    criterion = nn.MSELoss()  # Mean Squared Error Loss for regression tasks

    with torch.no_grad():  # Disable gradient calculation
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs, _ = model(inputs)
            
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            
            # Collect predictions and labels for correlation calculation
            all_predictions.extend(outputs.cpu().numpy().flatten())
            all_labels.extend(labels.cpu().numpy().flatten())

    test_loss = running_loss / len(dataloader.dataset)
    
    # Calculate correlation coefficients
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    
    rmse = np.sqrt(np.mean((all_predictions - all_labels) ** 2))
    plcc, _ = pearsonr(all_predictions, all_labels)
    srcc, _ = spearmanr(all_predictions, all_labels)
    krcc, _  = kendalltau(all_predictions, all_labels) # this is Kendall's Tau correlation coefficient
    
    return {
        'rmse': rmse,
        'plcc': plcc,
        'srcc': srcc,
        'krcc': krcc
    }

# Setup

In [5]:
IMAGENET_TRANSFORM = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

DENSENET_TRANSFORM = transforms.Compose([
    transforms.Resize((320, 320)),
    transforms.CenterCrop(300),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

ANNOTATION_FILE =  '../Dataset/AIGCIQA2023/real_images_annotations.csv'
BATCH_SIZE = 64
NUM_WORKERS = 20


# Create the datasets
imageNet_dataset = ImageAuthenticityDataset(csv_file=ANNOTATION_FILE, transform=IMAGENET_TRANSFORM)
denseNet_dataset = ImageAuthenticityDataset(csv_file=ANNOTATION_FILE, transform=DENSENET_TRANSFORM)

# Set random seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42) # Use torch.cuda.manual_seed_all(42) if using multiple GPUs
np.random.seed(42)
# Ensure generator state is consistent if using random_split multiple times with same seed desired
generator = torch.Generator().manual_seed(42)

# --- Corrected Splitting Logic ---

# Calculate split sizes for ImageNet dataset
imagenet_total_size = len(imageNet_dataset)
imagenet_train_size = int(0.7 * imagenet_total_size)
imagenet_val_size = int(0.2 * imagenet_total_size)
imagenet_test_size = imagenet_total_size - imagenet_train_size - imagenet_val_size

# Split the ImageNet dataset using distinct variable names
imagenet_train_ds, imagenet_val_ds, imagenet_test_ds = random_split(
    imageNet_dataset,
    [imagenet_train_size, imagenet_val_size, imagenet_test_size],
    generator=generator # Use the generator for consistent splits across datasets if needed
)

# Calculate split sizes for DenseNet dataset (assuming same proportions on the same base data length)
# Note: len(imageNet_dataset) should equal len(denseNet_dataset) if ANNOTATION_FILE is the same
densenet_total_size = len(denseNet_dataset)
densenet_train_size = int(0.7 * densenet_total_size)
densenet_val_size = int(0.2 * densenet_total_size)
densenet_test_size = densenet_total_size - densenet_train_size - densenet_val_size

# Split the DenseNet dataset using distinct variable names
# It's crucial that the indices corresponding to train/val/test are the same
# across both splits if you intend to compare models fairly on the *exact* same subsets.
# random_split with the same generator ensures this if the lengths are the same.
densenet_train_ds, densenet_val_ds, densenet_test_ds = random_split(
    denseNet_dataset,
    [densenet_train_size, densenet_val_size, densenet_test_size],
    generator=generator # Reuse the same generator
)

# --- Corrected DataLoader Creation ---

# Create DataLoaders for ImageNet-compatible models using the correct splits
imagenet_train_dataloader = DataLoader(
    imagenet_train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS
)
imagenet_val_dataloader = DataLoader(
    imagenet_val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)
imagenet_test_dataloader = DataLoader(
    imagenet_test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)

# Create DataLoaders for DenseNet models using the correct splits
denseNet_train_dataloader = DataLoader(
    densenet_train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS
)
denseNet_val_dataloader = DataLoader(
    densenet_val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)
denseNet_test_dataloader = DataLoader(
    densenet_test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)


# Create dictionaries containing the correctly assigned data loaders
imageNet_dataloaders = {
    'train': imagenet_train_dataloader,
    'val': imagenet_val_dataloader,
    'test': imagenet_test_dataloader
}

denseNet_dataloaders = {
    'train': denseNet_train_dataloader,
    'val': denseNet_val_dataloader,
    'test': denseNet_test_dataloader
}

# Optional: Verify lengths to be sure
print(f"ImageNet splits: Train={len(imagenet_train_ds)}, Val={len(imagenet_val_ds)}, Test={len(imagenet_test_ds)}")
print(f"DenseNet splits: Train={len(densenet_train_ds)}, Val={len(densenet_val_ds)}, Test={len(densenet_test_ds)}")
print(f"ImageNet Test Dataloader length: {len(imageNet_dataloaders['test'].dataset)}")
print(f"DenseNet Test Dataloader length: {len(denseNet_dataloaders['test'].dataset)}")

ImageNet splits: Train=957, Val=273, Test=138
DenseNet splits: Train=957, Val=273, Test=138
ImageNet Test Dataloader length: 138
DenseNet Test Dataloader length: 138


# Models load

In [6]:

barlow_twins_base = BarlowTwinsAuthenticityPredictor(freeze_backbone=True)
# Load weights only for security and if the file only contains the state_dict
barlow_twins_base.load_state_dict(torch.load('../Models/BarlowTwins/Weights/BarlowTwins_real_authenticity_finetuned.pth', weights_only=True))

# barlow_twins_negative_impact = BarlowTwinsAuthenticityPredictor(freeze_backbone=True)
# barlow_twins_negative_impact.load_state_dict(torch.load('../Models/BarlowTwins/Weights/real_authenticity_negative_impact_pruned_model.pth', weights_only=True))

barlow_twins_noise_out = BarlowTwinsAuthenticityPredictor(freeze_backbone=True)
barlow_twins_noise_out.load_state_dict(torch.load('../Models/BarlowTwins/Weights/real_authenticity_noise_out_pruned_model.pth', weights_only=True))

# DenseNet-161 Models
densenet161_base = DenseNet161AuthenticityPredictor(freeze_backbone=True)
densenet161_base.load_state_dict(torch.load('../Models/DenseNet-161/Weights/DenseNet-161_real_authenticity_finetuned.pth', weights_only=True))

# densenet161_negative_impact = DenseNet161AuthenticityPredictor(freeze_backbone=True)
# densenet161_negative_impact.load_state_dict(torch.load('../Models/DenseNet-161/Weights/real_authenticity_negative_impact_pruned_model.pth', weights_only=True))

densenet161_noise_out = DenseNet161AuthenticityPredictor(freeze_backbone=True)
densenet161_noise_out.load_state_dict(torch.load('../Models/DenseNet-161/Weights/real_authenticity_noise_out_pruned_model.pth', weights_only=True))

# EfficientNet-B3 Models
efficientnet_b3_base = EfficientNetB3AuthenticityPredictor(freeze_backbone=True)
efficientnet_b3_base.load_state_dict(torch.load('../Models/EfficientNet-B3/Weights/EfficientNetB3_real_authenticity_finetuned.pth', weights_only=True))

# efficientnet_b3_negative_impact = EfficientNetB3AuthenticityPredictor(freeze_backbone=True)
# efficientnet_b3_negative_impact.load_state_dict(torch.load('../Models/EfficientNet-B3/Weights/real_authenticity_negative_impact_pruned_model.pth', weights_only=True))

efficientnet_b3_noise_out = EfficientNetB3AuthenticityPredictor(freeze_backbone=True)
efficientnet_b3_noise_out.load_state_dict(torch.load('../Models/EfficientNet-B3/Weights/real_authenticity_noise_out_pruned_model.pth', weights_only=True))

# ResNet-152 Models
resnet152_base = ResNet152AuthenticityPredictor(freeze_backbone=True)
resnet152_base.load_state_dict(torch.load('../Models/ResNet-152/Weights/ResNet-152_real_authenticity_finetuned.pth', weights_only=True))

# resnet152_negative_impact = ResNet152AuthenticityPredictor(freeze_backbone=True)
# resnet152_negative_impact.load_state_dict(torch.load('../Models/ResNet-152/Weights/real_authenticity_negative_impact_pruned_model.pth', weights_only=True))

resnet152_noise_out = ResNet152AuthenticityPredictor(freeze_backbone=True)
resnet152_noise_out.load_state_dict(torch.load('../Models/ResNet-152/Weights/real_authenticity_noise_out_pruned_model.pth', weights_only=True))

# VGG19 Models
vgg19_base = VGG19AuthenticityPredictor(freeze_backbone=True)
vgg19_base.load_state_dict(torch.load('../Models/VGG19/Weights/VGG19_real_authenticity_finetuned.pth', weights_only=True))

# vgg19_negative_impact = VGG19AuthenticityPredictor(freeze_backbone=True)
# vgg19_negative_impact.load_state_dict(torch.load('../Models/VGG19/Weights/real_authenticity_negative_impact_pruned_model.pth', weights_only=True))

vgg19_noise_out = VGG19AuthenticityPredictor(freeze_backbone=True)
vgg19_noise_out.load_state_dict(torch.load('../Models/VGG19/Weights/real_authenticity_noise_out_pruned_model.pth', weights_only=True))

# VGG16 Models
vgg16_base = VGG16AuthenticityPredictor(freeze_backbone=True)
vgg16_base.load_state_dict(torch.load('../Models/VGG16/Weights/VGG-16_real_authenticity_finetuned.pth', weights_only=True))

vgg16_negative_impact = VGG16AuthenticityPredictor(freeze_backbone=True)
vgg16_negative_impact.load_state_dict(torch.load('../Models/VGG16/Weights/real_authenticity_negative_impact_pruned_model.pth', weights_only=True))

vgg16_noise_out = VGG16AuthenticityPredictor(freeze_backbone=True)
vgg16_noise_out.load_state_dict(torch.load('../Models/VGG16/Weights/real_authenticity_noise_out_pruned_model.pth', weights_only=True))



Using cache found in /home/icaro.redepaolini@unitn.it/.cache/torch/hub/facebookresearch_barlowtwins_main
/home/icaro.redepaolini@unitn.it/.conda/envs/myenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/icaro.redepaolini@unitn.it/.conda/envs/myenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/icaro.redepaolini@unitn.it/.cache/torch/hub/facebookresearch_barlowtwins_main


<All keys matched successfully>

# Test models

In [ ]:

def test_all_models(models, imageNet_dataloaders, denseNet_dataloaders, device='cuda'):
    """
    Tests all models on the test dataset and stores the results in a dictionary.

    Args:
        models (dict): Dictionary of model names and their corresponding model instances.
        imageNet_dataloaders (dict): Dictionary of data loaders for ImageNet-compatible models.
        denseNet_dataloaders (dict): Dictionary of data loaders for DenseNet models.
        criterion (nn.Module): The loss function.
        device (str): Device to use for testing ('cuda' or 'cpu'). Defaults to 'cuda'.

    Returns:
        dict: A dictionary containing the metrics for each model on the test dataset.
    """
    results = {}

    for model_name, model in models.items():
        
        # Select the appropriate dataloader based on the model name
        if 'DenseNet' in model_name:
            
            dataloader = denseNet_dataloaders['test']
        else:
            
            dataloader = imageNet_dataloaders['test']

        test_metrics = test_model(model, dataloader, device)
        results[model_name] = test_metrics
        
    return results

# Define the models in a dictionary
models = {
    'BarlowTwins_base': barlow_twins_base,
    # 'BarlowTwins_sd_base': barlow_twins_sd_base,
    # 'BarlowTwins_negative_impact': barlow_twins_negative_impact,
    'BarlowTwins_pruned': barlow_twins_noise_out,
    'DenseNet161_base': densenet161_base,
    # 'DenseNet161_negative_impact': densenet161_negative_impact,
    'DenseNet161_pruned': densenet161_noise_out,
    'EfficientNetB3_base': efficientnet_b3_base,
    # 'EfficientNetB3_negative_impact': efficientnet_b3_negative_impact,
    'EfficientNetB3_pruned': efficientnet_b3_noise_out,
    'ResNet152_base': resnet152_base,
    # 'ResNet152_negative_impact': resnet152_negative_impact,
    'ResNet152_pruned': resnet152_noise_out,
    'VGG19_base': vgg19_base,
    # 'VGG19_negative_impact': vgg19_negative_impact,
    'VGG19_pruned': vgg19_noise_out,
    'VGG16_base': vgg16_base,
    # 'VGG16_negative_impact': vgg16_negative_impact,
    'VGG16_pruned': vgg16_noise_out,

}


# Test all models and store the results
results = test_all_models(models, imageNet_dataloaders, denseNet_dataloaders, device='cuda')

# Print the results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    print(f"  RMSE: {metrics['rmse']:.4f}")
    print(f"  PLCC: {metrics['plcc']:.4f}")
    print(f"  SRCC: {metrics['srcc']:.4f}")
    print(f"  KRCC: {metrics['krcc']:.4f}")
    print()  # Print a newline for better readability
# Save the results to a markdown file
with open('model_results.md', 'w') as f:
    f.write("# Model Results\n\n")
    for model_name, metrics in results.items():
        f.write(f"## {model_name}\n")
        f.write(f"- RMSE: {metrics['rmse']:.4f}\n")
        f.write(f"- PLCC: {metrics['plcc']:.4f}\n")
        f.write(f"- SRCC: {metrics['srcc']:.4f}\n")
        f.write(f"- KRCC: {metrics['krcc']:.4f}\n\n")
    f.write("## Summary\n")
    f.write("| Model Name | RMSE | PLCC | SRCC | KRCC |\n")
    f.write("|------------|------|------|------|------|\n")
    for model_name, metrics in results.items():
        f.write(f"| {model_name} | {metrics['rmse']:.4f} | {metrics['plcc']:.4f} | {metrics['srcc']:.4f} | {metrics['krcc']:.4f} |\n")
    

Model: BarlowTwins_base
  RMSE: 7.0579
  PLCC: 0.6516
  SRCC: 0.5901
  KRCC: 0.4228

